In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly.express as px 
import cufflinks as cf
import plotly.graph_objects as go
import datetime as dt
from pandas_profiling import ProfileReport
# cf.go_offline()
%matplotlib inline 

# Dickey-Fuller test and other useful functions

In [2]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(ts:pd.DataFrame)-> int:
    try:
        dftest=adfuller(ts,autolag="AIC")
        dfoutput=pd.Series(dftest[0:4],index=["Test Statistic","p-value","#Lags Used",
                                             "Number of Observation Used"])
        for key,value in dftest[4].items():
            dfoutput["Critical Value (%s)"%key]=value
        return dfoutput["p-value"]  #Funzione che verifica la stazionaritò di un set di dati
    except:
        pass
def differentiate(ds:pd.DataFrame)-> list:
    dsdiff=ds.diff(periods=1)
    dsdiff.dropna(inplace=True)
    #STATIONARITY
    lg=0
    cv=0.01
    if test_stationarity(dsdiff) <=cv:
        print("Il Dataset dopo una diff è stazionario, quindi d è pari a 1")
        print("Il p-value dell' Augmented Dickey Fuller test è ",test_stationarity(dsdiff))
        d=1
        
    else:
        dsdiff=dsdiff.diff(periods=1)
        dsdiff.dropna(inplace=True)
        if test_stationarity(dsdiff) <=cv:
            print("Il Dataset dopo due diff è stazionario, quindi d è pari a 2")
            print("Il p-value dell' Augmented Dickey Fuller test è ",test_stationarity(dsdiff))
            d=2
            
        else:
            lg=1
            print("Il dataset non è stazionario con due differenziazioni,")
            print(" bisogna perciò usare il logaritmo per renderlo stazionario")
            dslog=np.log(ds)
            dslogdiff=dslog.diff(periods=1)
            dslogdiff.dropna(inplace=True)
            if test_stationarity(dslogdiff)<=cv:
                print("La serie logaritmica è stazionaria e d è pari a 1 ")
                print("Il p-value dell' Augmented Dickey Fuller test è ",test_stationarity(dslogdiff))
                d=1
                dsdiff=dslogdiff
                ds=dslog
                
            else:
                dslogdiff=dslogdiff.diff(periods=1)
                dslogdiff.dropna(inplace=True)
                if test_stationarity(dslogdiff)<=cv:
                    print("La serie logaritmica è stazionaria dopo due differenziazioni e d=2")
                    print("Il p-value dell' Augmented Dickey Fuller test è ",test_stationarity(dslogdiff))
                    d=2
                    dsdiff=dslogdiff
                    ds=dslog
                else:
                    print("Non posso renderlo stazionario")
                    return False
                   
    return [ds,lg,d]     #Se un dataset non è stazionario questa funzione lo rende tale

def mean_absolute_percentage_error(y_true: float, y_pred:float)->float: 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100  #Prese in input due serie di valori trova il loro MAPE

In [3]:
datitot=pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv") 
datitot=datitot[datitot.denominazione_regione.str.contains("Lazio")].loc[:,["data","totale_ospedalizzati"]] #Raccogliamo i dati del Lazio


# Arima Model Auto

## Heuristic algorithm for the search of the best lenght for the trainset

In [49]:
#[["2020-09-06","2020-11-01"],["2020-10-06","2020-12-01"],["2020-11-06","2021-01-01"],["2020-12-08","2021-02-02"]]
#[["2020-10-01","2020-11-01"],["2020-11-01","2020-12-01"],["2020-12-01","2021-01-01"],["2021-01-02","2021-02-02"]]
#[["2020-04-01","2020-04-30"],["2020-06-01","2020-06-30"],["2020-10-02","2020-10-30"],["2020-12-01","2021-01-01"],["2021-01-01","2021-01-30"],["2021-03-03","2021-04-01"]]
lista=[["2020-04-01","2020-04-30"],["2020-06-01","2020-06-30"],["2020-10-02","2020-10-30"],["2020-12-01","2020-12-30"],["2021-01-01","2021-01-30"],["2021-03-03","2021-04-01"]]#Lista per i train
def Arima(lista:list)->list: #Funzione che trova l'intervallo migliore per fare previsioni e l'intervallo migliore per il Train
    n=[] #Lista vuota per contenere gli archi temporali con MAPE minore per ogni periodo
    for t in lista:#Per ognuno degli elementi in lista valutiamo che intervallo previsionale restituisce il MAPE minore
        #FORMATTAZIONI DATASET
        dataset=datitot.copy()
        dataset.dropna(inplace=True)
        dataset["data"]=dataset.data.str.slice(0,10)
        dataset["data"]=pd.to_datetime(dataset["data"],format="%Y/%m/%d")
        ts=t[0] #Inizio periodo elemento lista t
        ts1=t[1]#Fine periodo elemento lista t
        ts=pd.to_datetime(ts)
        ts1=pd.to_datetime(ts1)
        print("Facciamo la previsione del periodo da {} a {}".format(ts,ts1))
        dataset=dataset[(dataset.data>=ts) & (dataset.data<=ts1)]
        dataset.set_index("data",inplace=True)
        dataset = dataset.astype('float64')

#         dataset=dataset.diff(periods=1)
#         dataset.dropna(inplace=True)
        #APPLICO LE FUNZIONI DESCRITTE NELLA SEZIONE 1
        listaStationarity=differentiate(dataset)  #Funzione che calcola la stazionarietà ed eventualmente rende i dati stazionari
        dataset=listaStationarity[0]
        lg=listaStationarity[1]
        d=listaStationarity[2]


        #Dopo aver trovato d, creo una lista di tuple (p,d,q), dove metto tutte le possibili combinazioni p,q (con d fissato)
        #dove p e q sono compresi in un range tra 0 e 1
        import itertools
        p=range(0,5)
        q=range(0,5)
        c=range(0,5)
        pdq = list(itertools.product(p,c,q)) #Prodotto cartesiano fra i possibili valori della lista 
        l=list(filter(lambda lmb:lmb[1]==d,pdq)) #Filtro quelli in cui ho il mio valore d ottenuto dalla stazionarietà

        tr=0.8
        te=0.2
        

        m={} #Dizionario che contiene tutti i MAPE associati a ogni intervallo temporale del relativo periodo
        listaBestOrder={} #Dizionario che associa a ogni intervallo temporale la tupla (p*,d*,q*) che da il valore AIC ottimo
        cont=0
        k=1
        it=0
        while k==1:   #In un periodo di lunghezza massima 30 giorni vedo come si comportano le previsioni nei sotto-intervalli
            cont+=1 #Contatore per diminuire sempre di più il dataset
            ds=dataset.copy().iloc[cont:,:] #Dataset che viene diminuito ogni iterazione
            if len(ds)<24:
                break #Se il dataset diventa troppo corto fermiamo il ciclo
            if it==0:               
                X=ds.totale_ospedalizzati.values
                train=X[0:int(len(X)*tr+1)] #Il Trainset è l' 80% e il testset su cui faremo le previsioni è il 20%
                test=X[int(len(X)*tr)+1:]
                it=1
                print(test)
            else:
                train=train[1:]
                
            from statsmodels.tsa.arima_model import ARIMA #Importo la funzione che presi (p,d,q) effettua le previsioni
            diz={}
            import warnings  #Per avere un risultato più pulito nascondo i warnings che potrebbero comparire in output
            warnings.filterwarnings('ignore')
            from numpy import linalg
            for par in l:
                try:
                    model_arima = ARIMA(train,order=par)
                    model_arima_fit = model_arima.fit()
                    predictions= model_arima_fit.forecast(steps=len(test))[0]
                    diz[par]=[model_arima_fit.aic]
#                 
                except ValueError:
                    continue
                except linalg.LinAlgError:
                    continue
                except ZeroDivisionError:
                    continue
            print("La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di {} giorni è {}".format(len(train),list(diz.keys())[list(diz.values()).index(min(diz.values()))],min(diz.values())[0]))   #(1.2.2),(4,2,4)
            ARp=min(diz,key=diz.get) #Tupla (p*,d*,q*) con valore di AIC minimo 
            model_arima = ARIMA(ds.totale_ospedalizzati.values,order=ARp) #Faccio le previsioni che avrei con (p*,d*,q*) ottimo
            model_arima_fit = model_arima.fit()
            predictions= model_arima_fit.forecast(steps=len(test))[0]
            if lg==1:#Se dovessi aver usato il log per la stazionarietà riporto i dati nell'ordine di grandezza corretto
                predictions=np.exp(predictions)
                test=np.exp(test)
                train=np.exp(train)
            MAPE=mean_absolute_percentage_error(test,predictions)
            if (len(train) in m and m[len(train)]>(MAPE)) or len(train) not in m: #Riempio i dizionari 
                m[len(train)]=MAPE
                listaBestOrder[len(train)]=ARp
        h=min(m,key=m.get)  #Intervallo che restituisce errore mape minimo, indicativo del periodo t che consideriamo
        print("The time window we consider, which returns the minimum MAPE for the period {}, is {} days. ".format(t,h))
        X=dataset.totale_ospedalizzati.values
        train=X[:h+1]
        ARp=listaBestOrder[h] #Prendo il parametro ARIMA ottimo che avevo trovato in precedenza
        #PREVISIONI
        model_arima = ARIMA(train,order=ARp)
        model_arima_fit = model_arima.fit()
        predictions= model_arima_fit.forecast(steps=len(test))[0] #Trovo le previsioni
        fig=go.Figure()
        #PLOTTING
        fig=fig.add_trace(go.Scatter(x=dataset.index[len(train):],y=predictions,hovertext="Predictions",name="Predictions"))
        fig=fig.add_trace(go.Scatter(x=dataset.index[0:len(train)+1],y=dataset.totale_ospedalizzati[:len(train)+1],hovertext="Train",name="Train"))
        fig=fig.add_trace(go.Scatter(x=dataset.index[len(train):],y=test,hovertext="Test",name="Test"))
        fig.update_layout(title="Forecasting and trend of the hospitalized from {} to {}".format(ts,ts1))
        fig.show()
        n.append(h) # h è uno degli intervalli candidati a essere la lunghezza ottima per il trainset
    pbestTrain1=int(round(sum(n)/len(n),0)) #Intervallo medio ottimo per fare previsioni
    print("The best interval for making prediction is ",pbestTrain1,"days")
    #FORMATTO IL DATASET PER LA PREVISIONE CHE FAREMO
    dataset1=datitot.copy()
    dataset1.dropna(inplace=True)
    dataset1["data"]=dataset1.data.str.slice(0,10)
    dataset1["data"]=pd.to_datetime(dataset1["data"],format="%Y/%m/%d")
    dataset1.set_index("data",inplace=True)

    return [pbestTrain1,dataset1]  
    



## Forecasting of the next week 

In [50]:
#USA QUESTA CELLA SE NON VUOI RUNNARE IL PROGRAMMA CON TUTTA L'ESECUZIONE DELLA SEZIONE 2.1
import datetime as datetime
rdataset=datitot.copy()
rdataset.dropna(inplace=True)
rdataset["data"]=rdataset.data.str.slice(0,10)
rdataset["data"]=pd.to_datetime(rdataset["data"],format="%Y/%m/%d")
rdataset.set_index("data",inplace=True)
tt=pd.to_datetime("2021-05-25")
dataset=rdataset.copy().loc[:tt]

# timend=pd.to_datetime("2021-06-01")
# timestart=pd.to_datetime("2020-02-25")
# dataset.rename(columns={"totale_ospedalizzati":"Hospitalized"},inplace=True)
# dataset.loc[:timend,:].diff(1).loc[timestart:,:]
# profile=ProfileReport(dataset)
# profile.to_file(output_file="ds.html")

In [51]:
#profile

In [52]:
listarima=Arima(lista)
# dataset=listarima[1]
pbestTrain=listarima[0]
##dataset=dataset.diff(periods=1)
##dataset.dropna(inplace=True)
gl=30-pbestTrain #30 è la lunghezza dei periodi presi per il train 
cont=0
pred={} #Dizionario che ha per chiavi i giorni che vogliamo predire, e per valori una lista che contiene le diverse predizioni per quel giorno
date=pd.date_range(start=dataset.index[-1],periods=gl+1)
date=date.astype(str)
for k in range(len(date)):#Inserisco una lista per ogni chiave di pred
    pred[date[k]]=[] 
while -gl+cont!=0: #Ciclo finchè la mia slicing window non raggiunge le gl previsioni
    dsfctry=dataset.iloc[-30+cont+1:,:]
    if cont!=gl-1:
        dsfctry=dsfctry.iloc[:-gl+cont+1]
    cont+=1
    teta=0
    while differentiate(dsfctry)==False:#Potrebbe capitare che la serie non riesce ad essere stazionaria, allora in quel caso aumentiamo il pbestTrain
        print("Non potevo rendere la serie stazionaria con {}, allora provo un intervallo di {} giorni...".format(len(dsfctry),len(dsfctry)+1))
        teta+=1
        if cont<gl-1:
            dsfctry=dataset.iloc[-30+cont-teta+1:-gl+cont+1,:] 
        else:
            dsfctry=dataset.iloc[-30+cont-teta+1:,:]
            
    dsfctry = dsfctry.astype('float64') 
    listaStationarity=differentiate(dsfctry)
    dsfctry=listaStationarity[0] #dsfctry è il dataset su cui andremo a prevedere i prossimi gl giorni
    lg=listaStationarity[1]
    d=listaStationarity[2]
    #PRODOTTO CARTESIANO
    import itertools
    p=range(0,5)
    q=range(0,5)
    c=range(0,5)
    pdq = list(itertools.product(p,c,q))
    l=list(filter(lambda lmb:lmb[1]==d,pdq)) #Filtro quelli in cui ho il mio valore d ottenuto dalla stazionarietà
    import warnings #Per avere un risultato più pulito nascondo i warnings che potrebbero comparire in output
    warnings.filterwarnings('ignore')
    from statsmodels.tsa.arima_model import ARIMA
    diz={}
    from numpy import linalg
    for k in l:
        try:
            model_arima =ARIMA(dsfctry,order=k)
            model_arima_fit = model_arima.fit()
            diz[k]=[model_arima_fit.aic]
        except ValueError:
            continue
        except linalg.LinAlgError:
            continue    
        except ZeroDivisionError:
            continue
    model_arima=ARIMA(dsfctry,order=min(diz,key=diz.get) )
    model_arima_fit=model_arima.fit()
    predictions=model_arima_fit.forecast(steps=gl+1)[0]
    if lg==1:#Se dovessi aver usato il log per la stazionarietà riporto i dati nell'ordine di grandezza corretto
        dsfctry=np.exp(dsfctry)
        predictions=np.exp(predictions)
    for k in range(len(predictions)): #Arrotondo le predizioni
        predictions[k]=round(predictions[k],0)
    predictions[0]=dsfctry.totale_ospedalizzati[-1]
    datex=pd.date_range(start=dsfctry.index[-1],periods=gl+1) #Date che saranno gli index delle previsioni
    datex=datex.astype(str)
    dfprevisionale=pd.DataFrame(predictions) #Dataframe che contiene le previsioni
    dfprevisionale.index=list(datex)
    for k in dfprevisionale.index:#Aggiungo le previsioni fatte per i giorni che sono chiavi per pred
        if k in pred:
            pred[k].append(dfprevisionale.loc[k,0])

fig=go.Figure()#Inizializzo un plot in cui inserisco la curva degli ospedalizzati fino a oggi, dopodichè aggiungerò le previsioni
fig=fig.add_trace(go.Scatter(x=dataset.iloc[-pbestTrain-1:,:].index,y=dataset.iloc[-pbestTrain-1:,:].totale_ospedalizzati,name="Train"))

print(pred)
for k in pred:#Faccio una media pesata delle previsioni, associando alle previsioni più recenti pesi più alti
    somma1=0
    somma2=0
    for j in range(len(pred[k])):
        somma1+=j+1
        somma2+=pred[k][j]*(j+1)
    pred[k]=round(somma2/somma1,0)
pred[str(dataset.index[-1])[:10]]=dataset.totale_ospedalizzati[-1] #Metto la data di oggi pari al valore che già conosco
#CREO IL DATAFRAME PREVISIONALE
dfprevisionale=pd.DataFrame(columns=["data","previsione"])
dfprevisionale["data"]=pred.keys()
dfprevisionale["data"]=pd.to_datetime(dfprevisionale["data"],format="%Y/%m/%d")
dfprevisionale.set_index("data",inplace=True)
dfprevisionale["previsione"]=pred.values()
display(dfprevisionale)
fig=fig.add_trace(go.Scatter(x=dfprevisionale.index,y=dfprevisionale.previsione,name="Forecastings")) #Aggiungo le previsioni a fig
fig=fig.add_trace(go.Scatter(x=dfprevisionale.index,y=rdataset.totale_ospedalizzati[tt:],name="Test"))
fig.show()
dsfcdiff=dfprevisionale.diff(periods=1) #Creo il dataframe che contiene gli incrementi di ospedalizzati giornalieri
dsfcdiff.dropna(inplace=True)
dsfcdiff.rename(columns={"previsione":"Nuovi_Ospedalizzati"},inplace=True)
graph=px.line(dsfcdiff,x=dsfcdiff.index,y=dsfcdiff.Nuovi_Ospedalizzati,title="Domanda dei nuovi ospedalizzati fino al {}".format(dsfcdiff.index[-1]))
graph.show()
display(dsfcdiff)
dsfcdiff.to_excel("Previsioni_settimanali.xlsx") #Stampo su Excel i dati che serviranno poi agli ospedali
dfprevisionale.to_excel("Prev._totali.xlsx")

Facciamo la previsione del periodo da 2020-04-01 00:00:00 a 2020-04-30 00:00:00
Il Dataset dopo due diff è stazionario, quindi d è pari a 2
Il p-value dell' Augmented Dickey Fuller test è  5.620559600606443e-19
[1603. 1607. 1600. 1575. 1532.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 24 giorni è (0, 2, 4)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (0, 2, 4)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (0, 2, 4)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (0, 2, 4)
The time window we consider, which returns the minimum MAPE for the period ['2020-04-01', '2020-04-30'], is 24 days. 


Facciamo la previsione del periodo da 2020-06-01 00:00:00 a 2020-06-30 00:00:00
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  0.004002144213625817
[198. 198. 200. 203. 202.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 24 giorni è (0, 1, 3)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (0, 1, 3)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (0, 1, 3)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (0, 1, 3)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (0, 1, 3)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (0, 1, 3)
The time window we consider, which returns the minimum MAPE for the period ['2020-06-01', '2020-06-30'], is 21 days. 


Facciamo la previsione del periodo da 2020-10-02 00:00:00 a 2020-10-30 00:00:00
Il Dataset dopo due diff è stazionario, quindi d è pari a 2
Il p-value dell' Augmented Dickey Fuller test è  1.9116158316118436e-06
[1757. 1798. 1835. 1954. 2048.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (0, 2, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (0, 2, 1)
The time window we consider, which returns the minimum MAPE for the period ['2020-10-02', '2020-10-30'], is 23 days. 


Facciamo la previsione del periodo da 2020-12-01 00:00:00 a 2020-12-30 00:00:00
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  2.79577795207859e-06
[3039. 3103. 3127. 3159. 3111.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 24 giorni è (0, 1, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (0, 1, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (0, 1, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (0, 1, 1)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (1, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (1, 1, 0)
The time window we consider, which returns the minimum MAPE for the period ['2020-12-01', '2020-12-30'], is 21 days. 


Facciamo la previsione del periodo da 2021-01-01 00:00:00 a 2021-01-30 00:00:00
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  2.0093082139905625e-07
[2827. 2791. 2747. 2727. 2689.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 24 giorni è (0, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (0, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (0, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (0, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (0, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (0, 1, 0)
The time window we consider, which returns the minimum MAPE for the period ['2021-01-01', '2021-01-30'], is 24 days. 


Facciamo la previsione del periodo da 2021-03-03 00:00:00 a 2021-04-01 00:00:00
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  0.000347057175278569
[3251. 3393. 3408. 3415. 3406.]
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 24 giorni è (4, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 23 giorni è (4, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 22 giorni è (4, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 21 giorni è (4, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 20 giorni è (4, 1, 0)
La tupla p,d,q che ha un valore minimo di Aic quando l'intervallo è di 19 giorni è (4, 1, 0)
The time window we consider, which returns the minimum MAPE for the period ['2021-03-03', '2021-04-01'], is 24 days. 


The best interval for making prediction is  23 days
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  4.1545967292963414e-24
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  4.1545967292963414e-24
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  0.002416683477830936
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  0.002416683477830936
Il dataset non è stazionario con due differenziazioni,
 bisogna perciò usare il logaritmo per renderlo stazionario
Non posso renderlo stazionario
Non potevo rendere la serie stazionaria con 23, allora provo un intervallo di 24 giorni...
Il Dataset dopo una diff è stazionario, quindi d è pari a 1
Il p-value dell' Augmented Dickey Fuller test è  0.001622234622648991
Il Dataset dopo una diff è stazionario, quindi d è pari a

,previsione
data,
2021-05-25,1361.0
2021-05-26,1250.0
2021-05-27,1196.0
2021-05-28,1143.0
2021-05-29,1091.0
2021-05-30,1060.0
2021-05-31,1009.0
2021-06-01,958.0


,Nuovi_Ospedalizzati
data,
2021-05-26,-111.0
2021-05-27,-54.0
2021-05-28,-53.0
2021-05-29,-52.0
2021-05-30,-31.0
2021-05-31,-51.0
2021-06-01,-51.0
